In [1]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

C:\Users\enhi\AppData\Local\Temp\ipykernel_15132\3827688388.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [57]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\VCI\NDVI-16\Trial\Tiles\Crop mask\Resampled to 300m'
Base_crop_mask = rxr.open_rasterio(main_dir  +'\\' + 'Base_crop_mask300.tif',masked=True)
Crop_layer = rxr.open_rasterio(main_dir+ '\\' + 'Cropland300.tif',masked=True)
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']

In [48]:
Phenology = Base_crop_mask.drop(labels = 1, dim = 'band')
Phenology_new = xr.where((Phenology > 36) & (Phenology <= 72), Phenology - 36, Phenology)
Phenology_new = xr.where(Phenology_new> 72 , Phenology_new - 72,Phenology_new)

In [49]:
count = 0
Phenology_month = Phenology_new
for i in range(0,12):
    Phenology_month = xr.where((Phenology_new > (0 + count)) & (Phenology_new <= (3 +count)), 1+i, Phenology_month)
    count = count+3
    

In [66]:
m = xr.concat([Crop_layer,Phenology_month], dim = 'band')

In [89]:
for month in range(0,12):
    Crop_mask = xr.where( (m[0].isnull() ==False) & ((m[1]==month+1) | (m[2] ==month+1) | ((m[1]>month+1) & (m[2]<m[1]) & (m[2]>month+1)) | ((m[1]<month+1) & (m[2]>month+1))), 1,0)
    Crop_mask = Crop_mask.where(Crop_mask!=0)
    
    Crop_mask.rio.to_raster(main_dir+'\\'+'Months'+ '\\' + 'Crop'+'\\'+ months[month]+ '.tif')